In [2]:
import csv
import numpy as np
import random
import time
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.cluster import MiniBatchKMeans
from imblearn.under_sampling import ClusterCentroids
from sklearn.utils import shuffle
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd


random.seed(10)

In [12]:
def reverse_one_hot(predictions):
    reversed_x = []
    for x in predictions:
        reversed_x.append(np.argmax(np.array(x)))
    return reversed_x
    
def learn_the_model_experiment(training_set_X, training_set_y, testing_set_X, testing_set_y, image_shape = (24, 300), num_classes = 2):
    batch_size = 100
    test_labels = tf.keras.utils.to_categorical(testing_set_y, num_classes)
    train_labels = tf.keras.utils.to_categorical(training_set_y, num_classes)
    
    train_images = training_set_X.reshape(training_set_X.shape[0], image_shape[0], image_shape[1], 1)
    test_images = testing_set_X.reshape(testing_set_X.shape[0], image_shape[0], image_shape[1], 1)
    
    model = Sequential()
    model.add(Conv2D(16, (3, 3), activation='relu', input_shape=(image_shape[0], image_shape[1], 1)))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss=tf.keras.losses.categorical_crossentropy, optimizer=tf.keras.optimizers.Adadelta(), metrics=['accuracy', 'mae', 'mse'])
    
    train_data_size = train_images.shape[0]
    test_data_size = test_images.shape[0]
    
    print("model will be trained with {} and be tested with {} sample".format(train_data_size, test_data_size))
    print("Fitting model to the training data...")
    model.fit(train_images, train_labels, batch_size=batch_size, epochs=5, verbose=1, validation_data=None)
    
    predictions_test = model.predict(test_images, batch_size=batch_size, verbose=1)
    predictions_train = model.predict(train_images, batch_size=batch_size, verbose=1)
    print(model.evaluate(test_images, test_labels, batch_size=batch_size, verbose=1))
    return predictions_test, predictions_train

def learn_and_test(X_resampled_train3, y_resampled_train3, X_test3, y_test3, image_shape = (24, 300), num_classes=2, save_confision_matrix=False):
    X_resampled_train3, y_resampled_train3 = shuffle(X_resampled_train3, y_resampled_train3)

    st = time.time()
    predictions_full_CC_test3, predictions_full_CC_train3 = learn_the_model_experiment(X_resampled_train3, y_resampled_train3, X_test3, y_test3, image_shape=image_shape, num_classes=num_classes)

    elapsed_time = time.time() - st
    print('Training model time (full):', elapsed_time/60, 'minutes')

    print("Evaluation accuracy score (full, test) = ", accuracy_score(y_test3, reverse_one_hot(predictions_full_CC_test3)))
    print("Evaluation accuracy score (full, train) = ", accuracy_score(y_resampled_train3, reverse_one_hot(predictions_full_CC_train3)))
        
    cf_matrix3 = confusion_matrix(y_test3, reverse_one_hot(predictions_full_CC_test3))
    if num_classes == 2:
        depict_confusion_matrix(cf_matrix3, 'Testing set (full)', save=save_confision_matrix, filename='/home/umcg-asorova/project/images/conf_mat_test.png')
    else:
        print(cf_matrix3)
    
    cf_matrix4 = confusion_matrix(y_resampled_train3, reverse_one_hot(predictions_full_CC_train3))
    if num_classes == 2:
        depict_confusion_matrix(cf_matrix4, 'Training set (full)', save=save_confision_matrix, filename='/home/umcg-asorova/project/images/conf_mat_train.png')
    else:
        print(cf_matrix4)
    return predictions_full_CC_test3, predictions_full_CC_train3

In [17]:
def read_data_from_file(filename):
    with open(filename, newline='') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',', quoting = csv.QUOTE_NONNUMERIC)
        data = [row for row in spamreader]
    return np.array(data)

def depict_confusion_matrix(cf_matrix, title, save=False, filename='confusion_matrix.png'):
    group_names = ['True Neg','False Pos','False Neg','True Pos']
    group_counts = ['{0:0.0f}'.format(value) for value in
                    cf_matrix.flatten()]
    group_percentages = ['{0:.2%}'.format(value) for value in
                         cf_matrix.flatten()/np.sum(cf_matrix)]
    labels = [f'{v1}\n{v2}\n{v3}' for v1, v2, v3 in
              zip(group_names,group_counts,group_percentages)]
    labels = np.asarray(labels).reshape(2,2)
    sns.heatmap(cf_matrix, annot=labels, fmt='', cmap='Blues')
    plt.title(title)
    if save:
        plt.savefig(filename, dpi=200)
    else:
        plt.show()
    plt.clf()

In [5]:
# takes 2 minutes to read all three datasets
# try classification with only first lead
waves = read_data_from_file('waves_full_1lead.csv')
print(waves.shape)

labels2classes = read_data_from_file('labels_full_2_classes_1lead.csv')[0]
print(labels2classes.shape)

labels3classes = read_data_from_file('labels_full_3_classes_1lead.csv')[0]
print(labels2classes.shape)

(44042, 600)
(44042,)
(44042,)


In [6]:
X_train, X_test, y_train, y_test = train_test_split(waves, labels2classes, train_size=0.75, stratify=labels2classes)

print('Original dataset shape (full):', Counter(labels2classes))
print('Resampled dataset shape (train):', Counter(y_train))
print('Resampled dataset shape (test):', Counter(y_test))

Original dataset shape (full): Counter({1.0: 22236, 0.0: 21806})
Resampled dataset shape (full): Counter({1.0: 16677, 0.0: 16354})
Resampled dataset shape (full): Counter({1.0: 5559, 0.0: 5452})


In [7]:
cc = ClusterCentroids(
    estimator=MiniBatchKMeans(n_init=1), sampling_strategy='not minority'
)
st = time.time()
X_resampled_train, y_resampled_train = cc.fit_resample(X_train, y_train)
elapsed_time = time.time() - st
print('Undersampling time (full):', elapsed_time/60, 'minutes')

print('Original dataset shape (train):', Counter(y_train))
print('Resampled dataset shape (train):', Counter(y_resampled_train))

Undersampling time (full): 2.72137770652771 minutes
Original dataset shape (full): Counter({1.0: 16677, 0.0: 16354})
Resampled dataset shape (full): Counter({0.0: 16354, 1.0: 16354})


In [20]:
pred_test, pred_train = learn_and_test(X_resampled_train, y_resampled_train, X_test, y_test, image_shape = (6, 100), num_classes=2, save_confision_matrix=True)

model will be trained with 32708 and be tested with 11011 sample
Fitting model to the training data...
Epoch 1/5
328/328 [==============================] - 4s 11ms/step - loss: 0.6982 - accuracy: 0.5027 - mae: 0.4995 - mse: 0.2525
Epoch 2/5
328/328 [==============================] - 4s 11ms/step - loss: 0.6946 - accuracy: 0.5058 - mae: 0.4990 - mse: 0.2507
Epoch 3/5
328/328 [==============================] - 4s 11ms/step - loss: 0.6927 - accuracy: 0.5113 - mae: 0.4985 - mse: 0.2498
Epoch 4/5
328/328 [==============================] - 4s 11ms/step - loss: 0.6921 - accuracy: 0.5194 - mae: 0.4983 - mse: 0.2495
Epoch 5/5
111/111 [==============================] - 1s 4ms/step - loss: 0.6897 - accuracy: 0.5806 - mae: 0.4983 - mse: 0.2483
[0.6897495985031128, 0.5806012153625488, 0.4982532560825348, 0.24830204248428345]
Training model time (full): 0.3655096332232157 minutes
Evaluation accuracy score (full, test) =  0.5806012169648533
Evaluation accuracy score (full, train) =  0.584505319799437

<Figure size 640x480 with 0 Axes>

In [ ]:
# remove obese, underweight and overweigth patients from the dataset in order to eliminate the differences 
# in the way the electrodes read the heart signals

waves = read_data_from_file('waves_full_1lead_normalBMI.csv')
print(waves.shape)

labels2classes = read_data_from_file('labels_full_2_classes_1lead_normalBMI.csv')[0]
print(labels2classes.shape)

labels3classes = read_data_from_file('labels_full_3_classes_1lead_normalBMI.csv')[0]
print(labels2classes.shape)

X_train, X_test, y_train, y_test = train_test_split(waves, labels2classes, train_size=0.75, stratify=labels2classes)

print('Original dataset shape (full):', Counter(labels2classes))
print('Resampled dataset shape (full):', Counter(y_train))
print('Resampled dataset shape (full):', Counter(y_test))

cc = ClusterCentroids(
    estimator=MiniBatchKMeans(n_init=1), sampling_strategy='not minority'
)
st = time.time()
X_resampled_train, y_resampled_train = cc.fit_resample(X_train, y_train)
elapsed_time = time.time() - st
print('Undersampling time (full):', elapsed_time/60, 'minutes')

print('Original dataset shape (full):', Counter(y_train))
print('Resampled dataset shape (full):', Counter(y_resampled_train))

pred_test, pred_train = learn_and_test(X_resampled_train, y_resampled_train, X_test, y_test, image_shape = (6, 100), num_classes=2)

In [ ]:
# use feature extraction and include all the patients since the wavelet transform removes all the noises
# remove obese, underweight and overweigth patients from the dataset in order to eliminate the differences 
# in the way the electrodes read the heart signals

waves = read_data_from_file('waves_full.csv')
print(waves.shape)

labels2classes = read_data_from_file('labels_full_2_classes.csv')[0]
print(labels2classes.shape)

labels3classes = read_data_from_file('labels_full_3_classes.csv')[0]
print(labels2classes.shape)
